In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.float_format = '{:,.2f}'.format

seed=42
np.random.seed = 42

### Cleaning

In [2]:
dataset = pd.read_csv('data/processed/train.csv.gzip',compression='gzip', sep =';',decimal=',')

# filtro de apenas apartamentos e vendas como está no enunciado
dataset = dataset[dataset['unitTypes']=='APARTMENT']
dataset = dataset[dataset['pricingInfos_businessType']=='SALE']

dataset.sample(1)

/home/willy/git/imoveis/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10,13,22,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,usableAreas,description,title,createdAt,publisherId,unitTypes,listingStatus,id,parkingSpaces,updatedAt,owner,images,address_country,address_zipCode,address_city,address_streetNumber,address_zone,address_geoLocation_precision,address_geoLocation_location_lon,address_geoLocation_location_lat,address_street,address_locationId,address_district,address_unitNumber,address_state,address_neighborhood,suites,publicationType,bathrooms,totalAreas,bedrooms,pricingInfos_price,pricingInfos_businessType,pricingInfos_period,pricingInfos_yearlyIptu,pricingInfos_rentalTotalPrice,pricingInfos_monthlyCondoFee
6389,63.0,"com 63m² de área útil, a 5 minutos da Av. Paul...",Centralo Park - Bela vista,2017-07-13T19:30:11.298Z,51dd26327f,APARTMENT,ACTIVE,9ca17c34db,2.0,2018-11-08T13:46:42.385Z,False,['https://s3-sa-east-1.amazonaws.com/vr.images...,BR,"1,410,001.00",São Paulo,523,NaN,ROOFTOP,-46.660294,-23.562403,Alameda Ministro Rocha Azevedo,BR>Sao Paulo>NULL>Sao Paulo>Centro>Cerqueira C...,NaN,NaN,São Paulo,Cerqueira César,1.0,STANDARD,2.0,63.0,2.0,840000,SALE,NaN,237.0,NaN,1798.0


Condomínios na ordem de 10^9 não fazem sentindo então temos um valor máximo na faixa de 2k
[fonte] https://imoveis.estadao.com.br/noticias/jardins-e-moema-sao-os-bairros-com-condominios-mais-caros-de-sao-paulo/
    
preço por metragem tatuapé
[fonte] https://www.proprietariodireto.com.br/preco-m2/tatuape-sao_paulo

In [3]:
def describe():
    """
    helper para achar valores estranhos nas pontas, máx e mín
    """
    num_lit = ['usableAreas','pricingInfos_price','parkingSpaces','suites','bathrooms'
    ,'totalAreas','bedrooms','pricingInfos_yearlyIptu','pricingInfos_monthlyCondoFee']

    return dataset[num_lit].describe()

In [4]:
#garantindo o float dtype
dataset = dataset[~dataset['usableAreas'].astype(str).str.contains('[A-Za-z]', na=False)]  
dataset['usableAreas'] = dataset['usableAreas'].astype('float32')

dataset = dataset[~dataset['pricingInfos_price'].astype(str).str.contains('[A-Za-z]', na=False)]    
dataset['pricingInfos_price'] = dataset['pricingInfos_price'].astype('float')

dataset['parkingSpaces']=dataset['parkingSpaces'].astype('float32')
dataset['suites']=dataset['suites'].astype('float')
dataset['bathrooms']=dataset['bathrooms'].astype('float')
dataset['totalAreas']=dataset['totalAreas'].astype('float')
dataset['bedrooms']=dataset['bedrooms'].astype('float')
dataset['pricingInfos_price']=dataset['pricingInfos_price'].astype('float')
dataset['pricingInfos_yearlyIptu']=dataset['pricingInfos_yearlyIptu'].astype('float')
dataset['pricingInfos_monthlyCondoFee']= dataset['pricingInfos_monthlyCondoFee'].astype('float')

#arrumando pontos na mão pois parece um misstype dado a metragem da casa, área e preço
#foi feito a partir do helper describe()

dataset.loc[91598,'usableAreas'] = 129 #misstype 
dataset.loc[3076,'pricingInfos_price'] = 7420000.00 #misstype 
dataset.loc[67986,'usableAreas'] = 90.00 #misstype 
dataset.loc[127253,'pricingInfos_price'] = 254349.20 #preço do m2 em tatuapé
dataset.loc[91030,'pricingInfos_price'] = 2240000.00 
dataset.loc[95790,'pricingInfos_price'] = 2072000.00
dataset.loc[16201,'usableAreas'] = 192.58 #misstype 
dataset.loc[16201,'totalAreas'] = 192.58 #misstype 
dataset.loc[28040,'totalAreas']=320.80 #misstype 
dataset.loc[120825,'totalAreas']=547.50 #misstype 
dataset.loc[10122,'bathrooms']=2 #mediana pois é área é de 65m²
dataset.loc[88364,'suites']=3 # #misstype 
dataset.loc[114508,'bedrooms']=2 #misstype 
dataset.loc[85067,'bedrooms']=3 #misstype 
dataset.loc[74140,'bedrooms']=2 #misstype 
dataset.loc[129094,'bedrooms']=1 #misstype 
dataset.loc[4875,'pricingInfos_yearlyIptu']=103670.00 #misstype 
dataset.loc[29837,'pricingInfos_yearlyIptu']=80499.99 #misstype 
dataset.loc[109411,'pricingInfos_yearlyIptu']=1610.00 #misstype 
dataset.loc[104816,'pricingInfos_monthlyCondoFee']=2443 #misstype 

"""
essa lista me pareceu que foi input errado, e como os valores estam perto da média
"""
bathLocs_list=[12128,34455,40067,71233,74140,87137,93715,100627,111310]
valor_list = [3,2,2,3,2,4,2,2,5]
dataset.loc[[x for x in bathLocs_list ],'bathrooms'] = [x for x in valor_list]

In [5]:
bathLocs_list = [5148,5642,22853,52435,56737,64387,104721]
dataset.loc[[x for x in bathLocs_list],'bathrooms']=4 #mediana de num de banheiros em casas dessa metragem (120m²+)

def usableArea(x):
    """
    garantindo que não haja totalAreas muito distantes da usableAreas
    """
    if x['totalAreas']*3 > x['usableAreas']:
        return x['usableAreas']
    else:
        return x['totalAreas']    
    
def usableArea2(x):
    """
    olhando o preço médio de um imóvel com mais de 1500m2, nos sites de imóvel,
    percebe-se que há poucos que custam menos de 2m
    """
    if x['usableAreas'] > 1500 and x['pricingInfos_price'] < 2000000:
        return x['usableAreas']/10
    else:
        return x['totalAreas']


def randomParkingSpaces():
    """
    como que a maioria dos aps tem entre 1 e 3 vagas, vamos achar a distribuição e 
    usar como probabilidade para trazer para um valor mais realista

    assim um >>dataset['parkingSpaces'].value_counts()<< pode nos dar uma boa aproximação
    """
    p = [0.051824852372388,0.473013194963949,0.286341859391663
        ,0.104668375061677,0.060483550066054,0.017317395387334
        ,0.006350772756936]
    values = [0,1,2,3,4,5,6]
    return np.random.choice(values,1,p=p,)[0]

def valueCondoFee1(x):
    if x['pricingInfos_monthlyCondoFee']>5000 and x['pricingInfos_price']>2000000:
        return x['pricingInfos_monthlyCondoFee']/10
    else:
        return x['pricingInfos_monthlyCondoFee']
    

def valueCondoFee2(x):
    if x['pricingInfos_monthlyCondoFee']>=10000 and x['usableAreas']<200:
        return x['pricingInfos_monthlyCondoFee']/100
    else:
        return x['pricingInfos_monthlyCondoFee']
    
def zeroUsableAreas(x):
    if x['usableAreas']==0:
        if pd.notna(x['usableAreasFromTitle']):
            return x['usableAreasFromTitle'] 
        elif pd.notna(x['usableAreasFromDescription']):
            return x['usableAreasFromDescription'] 
        else:
            return x['usableAreas']
    else:
        return x['usableAreas']

In [6]:
def proceduralFloat(dataset, *x):
    
    dataset['pricingInfos_monthlyCondoFee'] = dataset['pricingInfos_monthlyCondoFee'].apply(lambda x: x/1000 if x > 1000000 else x)
    dataset['totalAreas'] = dataset[['totalAreas','usableAreas']].apply(usableArea, axis=1)
    dataset['usableAreas']=dataset[['usableAreas','totalAreas','pricingInfos_price']].apply(usableArea2, axis=1)
    dataset['parkingSpaces'] = dataset['parkingSpaces'].apply(lambda x: randomParkingSpaces() if x>29 else x)
    dataset['pricingInfos_monthlyCondoFee'] = dataset[['pricingInfos_monthlyCondoFee','pricingInfos_price']].apply(valueCondoFee1,axis =1)
    dataset['pricingInfos_monthlyCondoFee'] = dataset[['pricingInfos_monthlyCondoFee','usableAreas']].apply(valueCondoFee2,axis =1)
    dataset['pricingInfos_yearlyIptu'] = dataset['pricingInfos_yearlyIptu'].apply(lambda x: x/10 if x>100000 else x )
    dataset['pricingInfos_price'] = dataset['pricingInfos_price'].apply(lambda x: x/10 if x > 10000000 else x)
    
    return 'part one, is done'

In [7]:
def extractInfoText(dataset, *x):

    dataset[['usableAreasFromTitle','a']] = dataset['title'].str.extract('(\d{1,4}\s?(m²|metros quadrados|metros quadrado),?)', expand=False)
    dataset['usableAreasFromTitle'] = dataset['usableAreasFromTitle'].str.extract(r'(\d+)', expand=False)

    dataset[['usableAreasFromDescription','a']] = dataset['description'].str.extract('(\d{1,4}\s?(m²|metros quadrados|metros quadrado),?)', expand=False)
    dataset['usableAreasFromDescription'] =dataset['usableAreasFromDescription'].str.extract(r'(\d+)', expand=False)

    dataset[['vagaFromTitle', 'b']]= dataset['title'].str.extract('(\s?\d{1,2}\s?(vaga|vagas|estacionamento))', expand=False)


    dataset['usableAreas']=dataset['usableAreas'].fillna(dataset['usableAreasFromDescription'])
    dataset['usableAreas']=dataset['usableAreas'].fillna(dataset['usableAreasFromTitle'])
    dataset['usableAreas']=dataset['usableAreas'].fillna(dataset['totalAreas'])

    dataset['usableAreas']= dataset[['usableAreas','usableAreasFromTitle','usableAreasFromDescription']].apply(zeroUsableAreas, axis=1 )
    dataset.usableAreas = dataset.usableAreas.astype('float')
    
    interesting_string='metrô|museu|avenida|av|parques|parque|padaria|perto|marginal|shopping|perto do metro'
    dataset['descriptionAndTitle']=dataset['description']+' '+dataset['title']
    dataset['interestingFlag']=dataset['descriptionAndTitle'].str.contains(interesting_string, regex=True)
    
    return 'part two, is done'

proceduralFloat(dataset, usableArea,usableArea2,randomParkingSpaces,valueCondoFee1,valueCondoFee2)
extractInfoText(dataset,zeroUsableAreas)

'done2'

In [9]:
dataset.to_csv('data/processed/train_cleaned.csv.gzip', sep =';', compression='gzip')

In [ ]:
usefullCollumns_list = ['id','usableAreas','totalAreas','parkingSpaces','address_zone','address_locationId'
,'address_neighborhood','suites','bedrooms','bathrooms','pricingInfos_price','pricingInfos_yearlyIptu'
,'pricingInfos_monthlyCondoFee']

In [ ]:
dataset[usefullCollumns_list].describe()

In [ ]:
dataset[dataset['pricingInfos_monthlyCondoFee']>100000]

In [ ]:
#amostra de 15%
_ = dataset[dataset['usableAreas']!=0]
sns.set_theme(style="darkgrid")
g = sns.jointplot(x="usableAreas", y="pricingInfos_price"
                  ,data=dataset.sample(round(_.shape[0]*.20), random_state=seed)
                  ,kind="reg", truncate=False
                  ,color="m", height=7)

In [ ]:
dataset.pricingInfos_price.describe()

In [ ]:
metro_df = dataset[dataset.title.str.contains('metrô', na=False)]

g = sns.jointplot(x="usableAreas", y="pricingInfos_price"
                  ,data=metro_df
                  ,kind="reg", truncate=False
                  ,color="m", height=7)